In [1]:
from scipy import stats
import pandas as pd
from statsmodels.stats.multitest import multipletests
import numpy as np

# Create a DataFrame with RNA sequencing data
data = {
    'Gene Name': ['Gene_I', 'Gene_II', 'Gene_III', 'Gene_IV', 'Gene_V',
                  'Gene_VI', 'Gene_VII', 'Gene_VIII', 'Gene_IX', 'Gene_X'],
    'Control_1': [190, 150, 165, 149, 752, 507, 150, 507, 1254, 752],
    'Control_2': [254, 50, 325, 154, 950, 1690, 752, 1120, 582, 480],
    'Disease_1': [1540, 964, 456, 540, 64, 215, 2150, 1840, 498, 52],
    'Disease_2': [1920, 823, 478, 1920, 82, 480, 1752, 990, 62, 157]
}

df = pd.DataFrame(data)

# Convert columns to numeric and handle non-numeric values
numeric_cols = ['Control_1', 'Control_2', 'Disease_1', 'Disease_2']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Calculate fold change and log2FC
df['Fold Change'] = (df['Disease_1'] + df['Disease_2']) / (df['Control_1'] + df['Control_2'])
df['Log2FC'] = np.log2(df['Fold Change'])

# Perform t-test and calculate p-values
t_statistic, p_value = stats.ttest_ind(df[['Control_1', 'Control_2']], df[['Disease_1', 'Disease_2']], axis=1)
df['P-value'] = p_value

# Adjust p-values using the Benjamini-Hochberg procedure
reject, p_adj, _, _ = multipletests(df['P-value'], method='fdr_bh')
df['Padjval'] = p_adj

# Determine Up/Down-regulated genes based on log2FC and adjusted p-values
df['Regulation'] = np.where((df['Log2FC'] > 0) & (df['Padjval'] < 0.05), 'Up', 
                            np.where((df['Log2FC'] < 0) & (df['Padjval'] < 0.05), 'Down', 'Not significant'))

# Print the final DataFrame
print(df[['Gene Name', 'Fold Change', 'Log2FC', 'P-value', 'Padjval', 'Regulation']])


   Gene Name  Fold Change    Log2FC   P-value   Padjval       Regulation
0     Gene_I     7.792793  2.962140  0.015936  0.053123  Not significant
1    Gene_II     8.935000  3.159468  0.011657  0.053123  Not significant
2   Gene_III     1.906122  0.930641  0.110761  0.184602  Not significant
3    Gene_IV     8.118812  3.021269  0.258474  0.323092  Not significant
4     Gene_V     0.085781 -3.543191  0.015937  0.053123  Not significant
5    Gene_VI     0.316340 -1.660450  0.341040  0.369781  Not significant
6   Gene_VII     4.325942  2.113014  0.053285  0.133211  Not significant
7  Gene_VIII     1.739398  0.798588  0.369781  0.369781  Not significant
8    Gene_IX     0.305011 -1.713067  0.252193  0.323092  Not significant
9     Gene_X     0.169643 -2.559427  0.072506  0.145012  Not significant
